# Hyperparameter Tuning Using Optuna

## Creating a Class for Easier Hyperparameter Tuning With Optuna

In [1]:
# package imports for OptunaTuner class
import logging
import optuna
import pandas as pd
from typing import Callable
from pickle import dump
%load_ext blackcellmagic

In [2]:
class OptunaTuner:
    def __init__(
        self,
        objective: Callable[[optuna.trial.Trial], float],
        pruner: optuna.pruners,
        n_trials: int = 100,
        seed: int = 123,
        study_timeout: int = 5 * 3600,
        direction: str = "minimize",
        study_name: str = None,
    ):
        """Set up the Optuna tuner.

        Args:
            objective (Callable[[optuna.trial.Trial], float]): Objective function to optimize.
            pruner (optuna.pruners): Pruner to use.
            n_trials (int, optional): Number of trials. Defaults to 100.
            seed (int, optional): Random seed to use. Defaults to 123.
            study_timeout (int, optional): Automatic termination after how many seconds. Defaults to 5*3600.
            direction (str, optional): Whether to minimize or maximize objective. Defaults to "minimize".
            study_name (str, optional): Name of the optuna study. Defaults to None.
            save_plots (bool, optional): Whether to save the plotly plots or not. Defaults to False.
        """
        self.objective = objective
        self.pruner = pruner
        self.n_trials = n_trials
        self.seed = seed
        self.study_timeout = study_timeout
        self.direction = direction
        self.study_name = study_name if study_name else "optuna_study"

        # Set up logging
        self.logger = logging.getLogger()
        self.logger.setLevel(logging.INFO)
        self.logger.addHandler(logging.FileHandler(f"{self.study_name}.log", mode="w"))
        optuna.logging.enable_propagation()
        optuna.logging.disable_default_handler()

    def run_study(self):
        """
        Runs the optimization study.
        """
        self.logger.info("Start optimization.")

        self.study = optuna.create_study(
            direction=self.direction,
            sampler=optuna.samplers.TPESampler(seed=self.seed),
            pruner=self.pruner,
        )

        self.study.optimize(
            self.objective,
            n_trials=self.n_trials,
            timeout=self.study_timeout,
            show_progress_bar=True,
        )

        # Save study results to CSV
        df = self.study.trials_dataframe()
        df.to_csv(f"results/{self.study_name}_results.csv")
        # Save best parameters to CSV and pickle
        self.logger.info(f"Best params: {self.study.best_params}")
        self.save_best_params()

    def save_best_params(self):
        """
        Saves the best parameters to a CSV file.
        """
        best_params = pd.DataFrame(
            self.study.best_params.items(), columns=["parameter", "value"]
        )
        best_params.to_csv(f"results/{self.study_name}_best_params.csv", index=False)
        with open(f"models/{self.study_name}_best_params.pkl", "wb") as f:
            dump(self.study.best_params, f)
        self.logger.info("Best parameters saved in csv and pkl format.")

    def plot_results(self, save_plots = False):
        """Generates and saves plotly plots of the study results."""

        # Plot optimization history
        optim_hist = optuna.visualization.plot_optimization_history(self.study)
        optim_hist.show()

        # Plot parameter importances
        p_import = optuna.visualization.plot_param_importances(self.study)
        p_import.show()

        # Plot parameter relationships
        contour = optuna.visualization.plot_contour(self.study)
        contour.show
        slice = optuna.visualization.plot_slice(self.study)
        slice.show()

        # Plot Timeline
        timeline = optuna.visualization.plot_timeline(self.study)
        timeline.show()

        # Parallel Coordinate Plot
        parallel = optuna.visualization.plot_parallel_coordinate(self.study)
        parallel.show()

        # Save plots
        if save_plots:
            optim_hist.write_image(f"{self.study_name}_optimization_history.png")
            p_import.write_image(f"{self.study_name}_param_importances.png")
            contour.write_image(f"{self.study_name}_contour.png")
            slice.write_image(f"{self.study_name}_slice.png")
            timeline.write_image(f"{self.study_name}_timeline.png")
            parallel.write_image(f"{self.study_name}_parallel.png")
            
            self.logger.info("Plots saved.")

## Elastic Net Example

In [3]:
# imports for lasso regression example
import numpy as np
from sklearn import datasets
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import optuna

In [4]:
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target


# Define the objective function for elastic net
def elastic_objective(trial):
    # Define cross-validation
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    scores = []
    # Suggest hyperparameter values
    params = {
        "alpha": trial.suggest_float("alpha", 0.0, 1.0, log=False),
        "l1_ratio": trial.suggest_float("l1_ratio", 0.0, 1.0),
        "positive": trial.suggest_categorical("positive", [True, False]),
        "max_iter": trial.suggest_int("max_iter", 100, 10000, log=True),
        "fit_intercept": False,
        "tol": trial.suggest_float("tol", 1e-5, 1e-3, log=True),
    }


    for step, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Standardize train and test sets
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)


        # Create a ElasticNet regressor with the suggested alpha
        regressor = ElasticNet(**params, random_state=42)

        # Fit the model on scaled training data
        regressor.fit(X_train_scaled, y_train)

        # Predict and calculate MSE
        y_pred = regressor.predict(X_test_scaled)
        score = mean_squared_error(y_test, y_pred)

        scores.append(score)

        # Report intermediate score to Optuna
        trial.report(np.mean(score), step)

        # Check for pruning
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # Return the average score over all folds
    return np.mean(scores)


# Create the pruner
pruner = optuna.pruners.MedianPruner(
    n_startup_trials=15, interval_steps=1
)

# Instantiate the tuner with the Lasso objective function
tuner = OptunaTuner(
    objective=elastic_objective,
    pruner=pruner,
    n_trials=100,
    seed=123,
    study_timeout=0.5 * 3600,
    direction="minimize",
    study_name="elastic_net_diabetes",
)

# Run the study
tuner.run_study()

# Plot the results
tuner.plot_results()

  0%|          | 0/100 [00:00<?, ?it/s]

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.331e+05, tolerance: 3.757e+02
  model = cd_fast.enet_coordinate_descent(
/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.467e+05, tolerance: 3.741e+02
  model = cd_fast.enet_coordinate_descent(
/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the 

[I 2024-03-30 15:13:16,995] Trial 0 finished with value: 26192.19415362568 and parameters: {'alpha': 0.6964691855978616, 'l1_ratio': 0.28613933495037946, 'positive': False, 'max_iter': 2744, 'tol': 7.01799283113844e-05}. Best is trial 0 with value: 26192.19415362568.
[I 2024-03-30 15:13:17,002] Trial 1 finished with value: 26302.637741241222 and parameters: {'alpha': 0.9807641983846155, 'l1_ratio': 0.6848297385848633, 'positive': True, 'max_iter': 484, 'tol': 0.00028714378103928377}. Best is trial 0 with value: 26192.19415362568.
[I 2024-03-30 15:13:17,009] Trial 2 finished with value: 26153.098784606816 and parameters: {'alpha': 0.4385722446796244, 'l1_ratio': 0.05967789660956835, 'positive': False, 'max_iter': 231, 'tol': 2.2433834533274747e-05}. Best is trial 2 with value: 26153.098784606816.
[I 2024-03-30 15:13:17,016] Trial 3 finished with value: 26099.09717111968 and parameters: {'alpha': 0.5315513738418384, 'l1_ratio': 0.5318275870968661, 'positive': False, 'max_iter': 2808, 'to

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.293e+06, tolerance: 1.769e+02
  model = cd_fast.enet_coordinate_descent(
/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.359e+06, tolerance: 1.762e+02
  model = cd_fast.enet_coordinate_descent(
/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the 

[I 2024-03-30 15:13:17,209] Trial 14 finished with value: 26045.68392490088 and parameters: {'alpha': 0.1753882956542422, 'l1_ratio': 0.7767377450242181, 'positive': False, 'max_iter': 189, 'tol': 1.0170803688691841e-05}. Best is trial 12 with value: 26042.58413262592.
[I 2024-03-30 15:13:17,236] Trial 15 finished with value: 26044.836157119757 and parameters: {'alpha': 0.1962357234564711, 'l1_ratio': 0.8160591907654154, 'positive': False, 'max_iter': 168, 'tol': 1.010115367316361e-05}. Best is trial 12 with value: 26042.58413262592.
[I 2024-03-30 15:13:17,256] Trial 16 finished with value: 26044.695223702336 and parameters: {'alpha': 0.20925744353235717, 'l1_ratio': 0.8414149107253112, 'positive': False, 'max_iter': 101, 'tol': 2.0584926956795804e-05}. Best is trial 12 with value: 26042.58413262592.
[I 2024-03-30 15:13:17,277] Trial 17 finished with value: 26062.148254369356 and parameters: {'alpha': 0.005978209644977167, 'l1_ratio': 0.8744399378075134, 'positive': False, 'max_iter': 

...

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.562e+02, tolerance: 2.228e+02
  model = cd_fast.enet_coordinate_descent(


[I 2024-03-30 15:13:17,845] Trial 44 pruned. 
[I 2024-03-30 15:13:17,869] Trial 45 finished with value: 26047.100932164918 and parameters: {'alpha': 0.05372851550525773, 'l1_ratio': 0.8334801682120019, 'positive': False, 'max_iter': 223, 'tol': 2.609326279062762e-05}. Best is trial 41 with value: 26037.579459370227.
[I 2024-03-30 15:13:17,888] Trial 46 pruned. 
[I 2024-03-30 15:13:17,905] Trial 47 pruned. 
[I 2024-03-30 15:13:17,924] Trial 48 finished with value: 26039.091446062826 and parameters: {'alpha': 0.17598634835350882, 'l1_ratio': 0.8966336870205862, 'positive': False, 'max_iter': 831, 'tol': 5.5367366961671404e-05}. Best is trial 41 with value: 26037.579459370227.
[I 2024-03-30 15:13:17,943] Trial 49 finished with value: 26038.717397972505 and parameters: {'alpha': 0.18277273989895781, 'l1_ratio': 0.9067130171467775, 'positive': False, 'max_iter': 888, 'tol': 6.056843657218431e-05}. Best is trial 41 with value: 26037.579459370227.
[I 2024-03-30 15:13:17,962] Trial 50 finished

/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.132e+05, tolerance: 9.549e+02
  model = cd_fast.enet_coordinate_descent(
/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.457e+05, tolerance: 9.510e+02
  model = cd_fast.enet_coordinate_descent(
/home/janhsc/miniconda3/envs/MachineLearning/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the 

[I 2024-03-30 15:13:18,048] Trial 55 pruned. 
[I 2024-03-30 15:13:18,069] Trial 56 finished with value: 26049.26827997214 and parameters: {'alpha': 0.07575889785977566, 'l1_ratio': 0.948525383168446, 'positive': False, 'max_iter': 1106, 'tol': 4.367803275625558e-05}. Best is trial 41 with value: 26037.579459370227.
[I 2024-03-30 15:13:18,086] Trial 57 pruned. 
[I 2024-03-30 15:13:18,106] Trial 58 finished with value: 26053.859017201317 and parameters: {'alpha': 0.035817065331952475, 'l1_ratio': 0.9024573008664311, 'positive': False, 'max_iter': 139, 'tol': 0.00011104328123218266}. Best is trial 41 with value: 26037.579459370227.
[I 2024-03-30 15:13:18,123] Trial 59 pruned. 
[I 2024-03-30 15:13:18,140] Trial 60 pruned. 
[I 2024-03-30 15:13:18,159] Trial 61 pruned. 
[I 2024-03-30 15:13:18,179] Trial 62 pruned. 
[I 2024-03-30 15:13:18,200] Trial 63 finished with value: 26035.155047234006 and parameters: {'alpha': 0.26311917699247456, 'l1_ratio': 0.9662650311645659, 'positive': False, 'max

/tmp/ipykernel_57369/2312430065.py:95: ExperimentalWarning:

plot_timeline is experimental (supported from v3.2.0). The interface can change in the future.

